STEAM_GAMES

Se importan las librerias a usar

In [221]:
# se importan las librerias que se van a usar 
import gzip
import ast
import chardet
import pandas as pd
import nltk

Se abre el archivo json comprimido que contiene la informacion de steam_game

In [222]:
# se define la ruta donde esta ubicado el archivo
file_path = '../datasets/steam_games.json.gz'
# se lee el archivo json
steam_games = pd.read_json(file_path, compression='gzip', lines=True)
steam_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [223]:
steam_games.head()


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


Se identifica las columnas que se van a usar para la creacion de las funciones solicitadas  para los endpoints, por lo cual se procede a hacer la limpieza de la Data. primero se eliminaran las filas que solo contienen valores nulos.

In [224]:
steam_games.dropna(how="all", inplace=True  )
steam_games.isnull().sum()

publisher       8052
genres          3283
app_name           2
title           2050
url                0
release_date    2067
tags             163
reviews_url        2
specs            670
price           1377
early_access       0
id                 2
developer       3299
dtype: int64

Para cambiar el tipo de dato en la columna id se identifico que contiene valores nulos, se procede a verificar.

In [225]:
steam_games[steam_games["id"].isnull()]

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


la fila 88384 solo contiene la url, parece no ser relevante para el analisis por ello se elimina, para la fila 119271 se filtrara para identificar si tiene un duplicado ya que contiene casi toda la informacion y nos puede servir para el analisis posterior

In [226]:
title = "Batman: Arkham City - Game of the Year Edition"
developer = "Rocksteady Studios,Feral Interactive (Mac)"
filtro = steam_games[(steam_games["title"]==title) & (steam_games["developer"]==developer)]
filtro

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
89378,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,200260.0,"Rocksteady Studios,Feral Interactive (Mac)"
119271,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",None,"[Single-player, Steam Achievements, Steam Trad...",19.99,0.0,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


se encontro otra fila con la misma informacion pero con su id, por ello se procede a eliminar la fila que no posee id

In [227]:
steam_games.dropna(subset=['id'], inplace=True) #elimina filas de la columna id que son null

se verifica que la columna id no tenga duplicados

In [228]:
steam_games[steam_games.duplicated(subset=['id'], keep=False)]


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


ya que la columna id no deberia contener duplicados se procede a eliminar una de las filas

In [229]:
steam_games.drop_duplicates(subset="id", inplace=True, keep="last")

con los cambios aplicados ya se puede cambiar el tipo de dato

In [230]:
steam_games["id"] = steam_games["id"].astype(int)

verificamos los nulos en el df

In [231]:
steam_games.isnull().sum()

publisher       8051
genres          3282
app_name           1
title           2049
url                0
release_date    2066
tags             162
reviews_url        0
specs            669
price           1377
early_access       0
id                 0
developer       3298
dtype: int64

Verificamos los nulos en la columana de genres que se usara mas adelante en las fuinciones, ya que la columna tags contiene casi la misma informacion verificamos los nulos que quedaran si cruzamos la informacion 

In [232]:
steam_games[(steam_games["genres"].isnull()) & (steam_games["tags"].isnull())].shape

(138, 13)

Se reemplaza los nulos por los datos de la columna tags ya que en algunas filas el contenido es igual y solo cambia el orden y en otras muy similar.

In [233]:
# Crea un conjunto de géneros únicos 
unique_genres = set(item for val in steam_games['genres'].dropna() for item in val)

# Filtra las etiquetas 'tags' 
steam_games['tags'] = steam_games['tags'].apply(lambda x: [item for item in x if item in unique_genres] if isinstance(x, list) else x) 

# Rellena valores nulos en la columna 'genres' 
steam_games['genres'].fillna(steam_games['tags'], inplace=True)

def add_tags_to_genres(row): 
    ''' Esta función añade valores de 'tags' a 'genres' que no estén ya'''
    genres = row['genres']
    tags = row['tags']
    if isinstance(tags, list) and isinstance(genres, list):
        for tag in tags:
            if tag not in genres:
                genres.append(tag)
    return genres

#aplico la funcion
steam_games['genres'] = steam_games.apply(lambda row: add_tags_to_genres(row), axis=1)

# elimina columna tags
steam_games.drop(columns=['tags'], inplace=True)

eliminamos los nulos de la columna genres ya que no se puede tratar

In [246]:
(steam_games['genres'].isnull().sum() / steam_games.shape[0])*100 #porcentaje de nulos

0.45831660644988625

In [ ]:
steam_games = steam_games[steam_games['genres'].notnull()]# se eliminan los nulos que quedaron en la columna genres

revisamos la columna release_date que es necesaria para hacer la primer funcion

In [241]:
# Contar la frecuencia de cada valor único en "release_date"
steam_games["release_date"].value_counts()

release_date
2012-10-16       100
2017-08-31        92
2017-09-26        89
2017-06-21        82
2017-07-25        78
                ... 
2017-05-28         1
December 2017      1
2010-01-21         1
2007-02-06         1
2018-10-01         1
Name: count, Length: 3506, dtype: int64

Extraemos el año que vamos a usar en la funcion, para ello creamos la columna year

In [235]:
# Función para extraer el año
def extraer_año(fecha):
    try:
        # Intenta extraer un año de la cadena
        año = pd.to_datetime(fecha, errors='coerce').year
        return int(año) if not pd.isnull(año) else None
    except:
        return None

# Aplicar la función para crear la nueva columna 'año'
steam_games['year'] = steam_games['release_date'].apply(extraer_año)



C:\Users\Willc\AppData\Local\Temp\ipykernel_13916\2240047720.py:5: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  año = pd.to_datetime(fecha, errors='coerce').year


In [243]:
steam_games['year'] = steam_games['year'].astype(int)# cambiamo la columna year a tipo entero que ocupa menos espacio de memoria

verificamos la proporcion de nulos, ya que no hay tratamiento para estos, si no son significativos se pueden eliminar

In [236]:
steam_games['year'].isnull().sum()

2240

In [237]:
(steam_games['year'].isnull().sum() / steam_games.shape[0])*100 #porcentaje de nulos

6.971243620067223

In [238]:
steam_games = steam_games[steam_games['year'].notnull()] #elimina los valore nulos

In [239]:
steam_games.isnull().sum() #verificcamos los nulos que quedan el el df

publisher       5996
genres           137
app_name           1
title              1
url                0
release_date       0
reviews_url        0
specs            669
price           1076
early_access       0
id                 0
developer       1250
year               0
dtype: int64

In [148]:
steam_games['developer'].fillna(steam_games['publisher'],inplace=True)

Se eliminan las columnas que no se usaran en el analisis

In [149]:
steam_games.drop(columns=['publisher','title','url','reviews_url','early_access','specs'], inplace=True)

In [150]:
steam_games.isnull().sum()

genres           138
app_name           1
release_date    2066
price           1377
id                 0
developer       3233
dtype: int64

In [245]:
steam_games.to_csv('../datasets/steam_games.csv',index=False)